#**🚢 Proyecto Titanic: Predicción de supervivencia con machine learning**
**Autor:** [Juan Camilo Sánchez Navarro ](https://linktr.ee/camiloarg30)

**Fecha:** Enero 2026

**Metodología:** CRISP-DM

#**1. Contexto**

## **1.1. Business understanding**

El hundimiento del Titanic es uno de los acontecimientos más importantes  y famosos que ha involucrado a naufragios en la historia. Aunque hubo un elemento de suerte, parece que ciertos grupos de personas tenían más probabilidades de sobrevivir que otros.

**Objetivo del Proyecto:**
Construir un modelo predictivo que responda a la pregunta: *"¿Qué tipo de personas tenían más probabilidades de sobrevivir?"* utilizando datos de los pasajeros como (nombre, edad, precio del boleto, clase socioeconómica, entre otros).

**Criterio de Éxito:**
Superar el benchmark base de Kaggle (Accuracy > 76%) mediante técnicas de análitica de datos.

#**2. Carga y revisión inicial**

## **2.1. Data Understanding**

Se importan las librerías a usar, se cargan los datasets.

*   **Train.csv:** Datos históricos con la variable objetivo `Survived` conocida.
*   **Test.csv:** Datos futuros donde debemos predecir la supervivencia.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

test = pd.read_csv("/kaggle/input/titanic/test.csv")
train = pd.read_csv("/kaggle/input/titanic/train.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


**Metadatos:**

El significado de cada una de las variables en los datasets es:

[descripción de cada una de las variables, definición y como se mide]

#**3. Diagnóstico de calidad de los datos**



Se realiza una inspección en cada dataset para detectar problemas de calidad en los datos

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


Se puede identificar que:
* Las variables ("Age", "Cabin") de **Train.csv:**
* Las variables ("Age", "Cabin", "Fare") de **Test.csv:**

tienen valores faltantes que deben ser tratados, debido a que sin datos completos no es posible implementar el modelo predictivo, para ello, se propone:
1. Imputar las variables "Age" para cada dataset y la variable "Fare" para el dataset **Test.csv:**
2. Eliminar la columna "Cabin" en cada dataset, ya que debido a que cuenta con muy pocos datos en comparación con el total de los datos. En terminos de porcentaje, se tiene:
* En **Train.csv:** 204/891 = 0.228956229
* En **Test.csv:** 91/417 = 0.2182254197

de tal manera que como en cada uno hay menos de un 30% de los datos, no se deben imputar porque al hacerlo, pueden verse sesgados los valores de la variables por solo un pequeña porción de los datos y por ello, se va a eliminar la variable "Cabin" tanto en **Train.csv:** como en **Test.csv:**

# **4. Limpieza y tratamiento de los datos**

## **4.1 Data Preparation**


###**Datos Faltantes:**

Basado en el diagnóstico anterior, se toman las siguientes decisiones estratégicas para manejar los datos faltantes:

1.  **Variable `Age` (Edad):** Imputar con la **Media**. La edad es vital para determinar supervivencia (niños/ancianos), y perder esos registros es muy costoso para el objetivo determinado.
2.  **Variable `Embarked` (Puerto):** Faltan solo 2 registros.
    *   *Acción:* Imputar con la **Moda** (el puerto más frecuente). El impacto estadístico es despreciable.
3.  **Variable `Cabin` (Cabina):** Faltan >70% de los datos.
    *   *Acción:* **Eliminar la variable**. Intentar imputar tantos datos introduciría un ruido masivo y sesgo artificial en el modelo.


In [5]:
#Limpieza de datos para train
train["Age"] = train["Age"].fillna(train["Age"].mean())

train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])


#Limpieza de datos para test
test["Age"] = test["Age"].fillna(test["Age"].mean())

test["Fare"] = test["Fare"].fillna(test["Fare"].mean())

test["Embarked"] = test["Embarked"].fillna(test["Embarked"].mode()[0])

#En ambos casos, se va a eliminar la variable "Cabin"
train = train.drop(columns=['Cabin'])
test = test.drop(columns=['Cabin'])

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 32.8+ KB


### 3.2 Codificación de Variables (Encoding)

Los modelos de Machine Learning requieren entradas numéricas. Las variables categóricas (`Sex`, `Embarked`) contienen información valiosa, pero están en formato texto.

**Estrategia de transformación:**
*   **Sex:** Mapeo binario directo (`male=0`, `female=1`).
*   **Embarked:** Mapeo numérico (`S=0`, `C=1`, `Q=2`).

**Selección de Variables (Feature Selection):**
Las variables `Name` y `Ticket` son identificadores únicos de texto y alta cardinalidad. De tal manera que no aportan valor predictivo directo, por lo que se decide **excluirlas** para simplificar el modelo.

In [7]:
for col in ["Sex", "Embarked"]:
    print(col, train[col].nunique(), train[col].unique())

Sex 2 ['male' 'female']
Embarked 3 ['S' 'C' 'Q']


In [8]:
#Para train.csv
train["Sex"] = train["Sex"].map({"male": 0, "female": 1}).astype("Int64")
train["Embarked"] = train["Embarked"].map({"S": 0, "C": 1, "Q":2}).astype("Int64")

#Para test.csv
test["Sex"] = test["Sex"].map({"male": 0, "female": 1}).astype("Int64")
test["Embarked"] = test["Embarked"].map({"S": 0, "C": 1, "Q":2}).astype("Int64")

train = train.drop(columns=['Name', 'Ticket'])
test = test.drop(columns=['Name', 'Ticket'])

train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    Int64  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    Int64  
dtypes: Int64(2), float64(2), int64(5)
memory usage: 64.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    Int64  
 3   Age          418 non-null    float64
 4   SibSp       